In [1]:
from lib import Bkend_res50_8top
from lib import Dataset_top_to_birdView
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import copy



model = Bkend_res50_8top()
t_in = torch.randn(1,3,224,224 )
t_in2 = torch.randn(1,4)
model(t_in,t_in2).shape


torch.Size([1, 4])

In [2]:
train_list = ['./data/train/000' ]
val_list   = ['./data/val/000' ]

training_generator     = Dataset_top_to_birdView( train_list )
validation_generator   = Dataset_top_to_birdView( val_list )

number of removed labels:  0
number of removed labels:  0


In [3]:
# training_generator[0]

train_loader = DataLoader(training_generator  ,batch_size=12 ,num_workers=12)
val_loader   = DataLoader(validation_generator,batch_size=12 ,num_workers=12)

In [4]:
from torch.utils.tensorboard import SummaryWriter
import torchvision
writer = SummaryWriter('runs/bird_eyeview_experiment_1')

# https://github.com/lanpa/tensorboardX

# get some random training images
dataiter = iter(train_loader)
label_front, crop_front ,label_top = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(crop_front)

writer.add_image('training_set_batches', img_grid)
writer.add_graph(model,  (crop_front ,label_front))
writer.close()


/home/ali/.virtualenvs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ali/.virtualenvs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ali/.virtualenvs/py37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ali/.virtualenvs/py37/lib/python3.7/site

In [5]:
training_generator.bboxs_f

array([6.93938350e-310, 4.94539712e-316, 2.84268732e-316, 2.84268693e-316])

In [6]:

#this is good for classification task
# https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

# # helper function
# def select_n_random(data, labels, n=100):
#     '''
#     Selects n random datapoints and their corresponding labels from a dataset
#     '''
#     assert len(data) == len(labels)

#     perm = torch.randperm(len(data))
#     return data[perm][:n], labels[perm][:n]

# # select random images and their target indices


# images, labels = select_n_random(training_generator.imgs_f, training_generator.bboxs_b)

# # get the class labels for each image
# class_labels = [classes[lab] for lab in labels]

# # log embeddings
# features = images.view(-1, 28 * 28)
# writer.add_embedding(features,
#                     metadata=class_labels,
#                     label_img=images.unsqueeze(1))
# writer.close()

In [7]:
# next(iter(train_loader))

# for data in tqdm(train_loader):
#     pass

In [8]:
label_front, crop_front ,label_top =  next(iter(train_loader))
crop_front.shape
model(crop_front,label_front) , label_top

(tensor([[ 0.1096,  0.0281,  0.0902, -0.0530],
         [ 0.1311,  0.0051,  0.0787, -0.0649],
         [ 0.1178,  0.0203,  0.0575, -0.0754],
         [ 0.1310,  0.0187,  0.0758, -0.0623],
         [ 0.1231,  0.0238,  0.0500, -0.0618],
         [ 0.0988,  0.0235,  0.0669, -0.0482],
         [ 0.1314,  0.0016,  0.0852, -0.0659],
         [ 0.1153,  0.0199,  0.0832, -0.0713],
         [ 0.1215,  0.0303,  0.0714, -0.0642],
         [ 0.1335,  0.0231,  0.0972, -0.0661],
         [ 0.1210,  0.0234,  0.0749, -0.0649],
         [ 0.1268,  0.0265,  0.0939, -0.0681]], grad_fn=<TanhBackward>),
 tensor([[0.0150, 0.0060, 0.1930, 0.4270],
         [0.0040, 0.2120, 0.1860, 0.6400],
         [0.0750, 0.1440, 0.1550, 0.3800],
         [0.0730, 0.3670, 0.1490, 0.5890],
         [0.3340, 0.1120, 0.3950, 0.1730],
         [0.2910, 0.0150, 0.3760, 0.2250],
         [0.2530, 0.0040, 0.3580, 0.2860],
         [0.2480, 0.0550, 0.3440, 0.3280],
         [0.2470, 0.0930, 0.3330, 0.3550],
         [0.2420, 0.152

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model = model.to(device)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model,device_ids=[0,1])
    model = nn.DataParallel(model)

Let's use 2 GPUs!


In [10]:
# r = tqdm(train_loader)


def train_(train_loader , epoch):
    model.train()
    running_loss = 0.0
    r = tqdm(train_loader)
    for i, data in enumerate(r, 0):
        # get the inputs; data is a list of [inputs, labels]
        label_front, crop_front ,label_top = data
        label_front =label_front.to(device)
        crop_front =crop_front.to(device)
        label_top =label_top.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(crop_front,label_front)
        loss = criterion(outputs, label_top)
        loss.backward()
        optimizer.step()
        
        batch_loss = copy.deepcopy(loss.item())
        batch_loss = batch_loss / label_front.shape[0]
        
        writer.add_scalar('training batch loss',
                        batch_loss ,
                        epoch * len(train_loader) + i)
        

        # print statistics
        running_loss += loss.item() / len(train_loader)
        
        r.set_description(f'([T/{epoch}](L: {running_loss:0.6f} , BL{batch_loss: 0.6f})')
        
#         writer.close()
        
    writer.add_scalar('training loss',
                        running_loss ,
                        epoch )
    writer.close()
        
    
    
def validation_ (val_loader , epoch):
    model.eval()
    running_loss = 0.0
    r2 = tqdm(val_loader)
    for i, data in enumerate(r2, 0):
        # get the inputs; data is a list of [inputs, labels]
        label_front, crop_front ,label_top = data
        label_front =label_front.to(device)
        crop_front =crop_front.to(device)
        label_top =label_top.to(device)

        # forward + backward + optimize
        outputs = model(crop_front,label_front)
        loss = criterion(outputs, label_top)
        
        batch_loss = copy.deepcopy(loss.item())
        batch_loss = batch_loss / label_front.shape[0]
        
        writer.add_scalar('validation batch loss',
                        batch_loss ,
                        epoch * len(val_loader) + i)
        

        # print statistics
        running_loss += loss.item() / len(val_loader)
        
        r2.set_description(f'[E/{epoch}](L: {running_loss:0.6f} , BL{loss.item() / label_front.shape[0]: 0.6f})')
        
    writer.add_scalar('validation loss',
                        running_loss ,
                        epoch )
    writer.close()
        
    return running_loss
        

In [11]:


criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
best_loss = 1000000000;

for epoch in range(10):  # loop over the dataset multiple times
    print(f'============ Epoch {epoch} ============')
    
    train_(train_loader , epoch)
    curr_val_loss = validation_ (val_loader , epoch)
    
    for name, param in model.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
    writer.close()
#     print('curr_val_loss' , curr_val_loss)
    
    

    if curr_val_loss  < best_loss:
        torch.save(model.state_dict(), f"./model_E{epoch:03d}_Loss{curr_val_loss:.6f}.pt")
        print (f"./model_E{epoch:03d}_Loss{curr_val_loss:.6f}.pt  is saved.")
        best_loss = curr_val_loss



print('Finished Training')

============ Epoch 0 ============



./model_E000_Loss0.149444.pt  is saved.
============ Epoch 1 ============



./model_E001_Loss0.114107.pt  is saved.
============ Epoch 2 ============



./model_E002_Loss0.090177.pt  is saved.
============ Epoch 3 ============



./model_E003_Loss0.073898.pt  is saved.
============ Epoch 4 ============


...


./model_E005_Loss0.055575.pt  is saved.
============ Epoch 6 ============



./model_E006_Loss0.050835.pt  is saved.
============ Epoch 7 ============



./model_E007_Loss0.047844.pt  is saved.
============ Epoch 8 ============



./model_E008_Loss0.046020.pt  is saved.
============ Epoch 9 ============



./model_E009_Loss0.044951.pt  is saved.
Finished Training


In [12]:
# !pip install --upgrade numpy

In [13]:


# # show images
# matplotlib_imshow(img_grid, one_channel=True)

# # write to tensorboard
# writer.add_image('four_fashion_mnist_images', img_grid)